In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<img src= "https://cdn.bulbagarden.net/upload/8/88/Mission_Guides.png"/>

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


# Probemos los modelos...

![img](https://img.pokemondb.net/artwork/melmetal.jpg)

In [3]:
test = np.array([[580.0,90.0,85.0,100.0,95.0,125.0,85.0]])

<img src="https://images.gameinfo.io/pokemon/256/493-11.png" weidth="400"/>

In [4]:
test2 = np.array([[720.0,120.0,120.0,120.0,120.0,120.0,120.0]])

## ¿Será legendario Melmetal.... y qué tal Arceus?

## ¿Será?

In [23]:
pokedex =  pd.read_csv("/content/drive/My Drive/Colab Notebooks/pokedex_(Update_05.20).csv")
pokedex

,Unnamed: 0,pokedex_number,name,german_name,japanese_name,generation,status,species,type_number,type_1,type_2,height_m,weight_kg,abilities_number,ability_1,ability_2,ability_hidden,total_points,hp,attack,defense,sp_attack,sp_defense,speed,catch_rate,base_friendship,base_experience,growth_rate,egg_type_number,egg_type_1,egg_type_2,percentage_male,egg_cycles,against_normal,against_fire,against_water,against_electric,against_grass,against_ice,against_fight,against_poison,against_ground,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy
0,0,1,Bulbasaur,Bisasam,フシギダネ (Fushigidane),1,Normal,Seed Pokémon,2,Grass,Poison,0.7,6.9,2,Overgrow,NaN,Chlorophyll,318.0,45.0,49.0,49.0,65.0,65.0,45.0,45.0,70.0,64.0,Medium Slow,2,Grass,Monster,87.5,20.0,1.0,2.0,0.5,0.5,0.25,2.0,0.5,1.0,1.0,2.0,2.0,1.00,1.00,1.0,1.0,1.0,1.0,0.5
1,1,2,Ivysaur,Bisaknosp,フシギソウ (Fushigisou),1,Normal,Seed Pokémon,2,Grass,Poison,1.0,13.0,2,Overgrow,NaN,Chlorophyll,405.0,60.0,62.0,63.0,80.0,80.0,60.0,45.0,70.0,142.0,Medium Slow,2,Grass,Monster,87.5,20.0,1.0,2.0,0.5,0.5,0.25,2.0,0.5,1.0,1.0,2.0,2.0,1.00,1.00,1.0,1.0,1.0,1.0,0.5
2,2,3,Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,Poison,2.0,100.0,2,Overgrow,NaN,Chlorophyll,525.0,80.0,82.0,83.0,100.0,100.0,80.0,45.0,70.0,236.0,Medium Slow,2,Grass,Monster,87.5,20.0,1.0,2.0,0.5,0.5,0.25,2.0,0.5,1.0,1.0,2.0,2.0,1.00,1.00,1.0,1.0,1.0,1.0,0.5
3,3,3,Mega Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,Poison,2.4,155.5,1,Thick Fat,NaN,NaN,625.0,80.0,100.0,123.0,122.0,120.0,80.0,45.0,70.0,281.0,Medium Slow,2,Grass,Monster,87.5,20.0,1.0,1.0,0.5,0.5,0.25,1.0,0.5,1.0,1.0,2.0,2.0,1.00,1.00,1.0,1.0,1.0,1.0,0.5
4,4,4,Charmander,Glumanda,ヒトカゲ (Hitokage),1,Normal,Lizard Pokémon,1,Fire,NaN,0.6,8.5,2,Blaze,NaN,Solar Power,309.0,39.0,52.0,43.0,60.0,50.0,65.0,45.0,70.0,62.0,Medium Slow,2,Dragon,Monster,87.5,20.0,1.0,0.5,2.0,1.0,0.50,0.5,1.0,1.0,2.0,1.0,1.0,0.50,2.00,1.0,1.0,1.0,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,1023,888,Zacian Hero of Many Battles,NaN,NaN,8,Legendary,Warrior Pokémon,1,Fairy,NaN,2.8,110.0,1,Intrepid Sword,NaN,NaN,670.0,92.0,130.0,115.0,80.0,115.0,138.0,NaN,NaN,NaN,Slow,1,Undiscovered,NaN,NaN,120.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,2.0,1.0,1.0,1.0,0.50,1.00,1.0,0.0,0.5,2.0,1.0
1024,1024,889,Zamazenta Crowned Shield,NaN,NaN,8,Legendary,Warrior Pokémon,2,Fighting,Steel,2.9,785.0,1,Dauntless Shield,NaN,NaN,720.0,92.0,130.0,145.0,80.0,145.0,128.0,NaN,NaN,NaN,Slow,1,Undiscovered,NaN,NaN,120.0,0.5,2.0,1.0,1.0,0.50,0.5,2.0,0.0,2.0,1.0,1.0,0.25,0.25,1.0,0.5,0.5,0.5,1.0
1025,1025,889,Zamazenta Hero of Many Battles,NaN,NaN,8,Legendary,Warrior Pokémon,1,Fighting,NaN,2.9,210.0,1,Dauntless Shield,NaN,NaN,670.0,92.0,130.0,115.0,80.0,115.0,138.0,NaN,NaN,NaN,Slow,1,Undiscovered,NaN,NaN,120.0,1.0,1.0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,2.0,2.0,0.50,0.50,1.0,1.0,0.5,1.0,2.0
1026,1026,890,Eternatus,NaN,NaN,8,Legendary,Gigantic Pokémon,2,Poison,Dragon,20.0,950.0,1,Pressure,NaN,NaN,690.0,140.0,85.0,95.0,145.0,95.0,130.0,NaN,NaN,NaN,Slow,1,Undiscovered,NaN,NaN,120.0,1.0,0.5,0.5,0.5,0.25,2.0,0.5,0.5,2.0,1.0,2.0,0.50,1.00,1.0,2.0,1.0,1.0,1.0


# Ejercicio

En esta sección deberán realizar un nuevo dataframe, pero considerando las siguientes características (ojo!, ya no son sólo de tipo normal y legendario...)

In [19]:
pokedex.status.unique()


array(['Normal', 'Sub Legendary', 'Legendary', 'Mythical'], dtype=object)

El objetivo ahora será realizar intercambio de la variable categórica "Status" por una nueva que le asigne un número en lugar de texto

In [14]:
lb_make = LabelEncoder()
pokedex.status=lb_make.fit_transform(pokedex.status)

Posteriormente, deberán hacer el procedimiento "normal" para los siguientes algoritmos
* Regresion Logística
* SVM (con kernel rbf - Gaussiano o radial)
* Decision Tree
* Random Forest

###Regresión Logística

In [33]:
X = np.array(pokedex[["total_points", "hp","attack","defense","sp_attack","sp_defense","speed"]]) #totalPoints-speed
y = np.array(pokedex['status'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 6)

In [51]:
modelReg = linear_model.LogisticRegression(max_iter=1200)
modelReg.fit(X_train,y_train)
y_pred = modelReg.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(modelReg.score(X_train, y_train))

print("Test 1: ", modelReg.predict(test))
print("Test 2: ", modelReg.predict(test2))


0.8980582524271845
0.9197080291970803
Test 1:  ['Normal']
Test 2:  ['Legendary']


###SVM

In [52]:
modelSVM = make_pipeline(StandardScaler(), SVC(gamma='auto'))
modelSVM.fit(X_train,y_train)

y_predSVM = modelSVM.predict(X_test)

print(accuracy_score(y_test, y_predSVM))
print(modelSVM.score(X_train, y_train))

print("Test 1: ", modelSVM.predict(test))
print("Test 2: ", modelSVM.predict(test2))

0.9029126213592233
0.9245742092457421
Test 1:  ['Normal']
Test 2:  ['Legendary']


###Decision Tree

In [53]:
modelTree = tree.DecisionTreeClassifier()
modelTree.fit(X_train,y_train)

y_predTree = modelTree.predict(X_test)

print(accuracy_score(y_test, y_predTree))
print(modelTree.score(X_train, y_train))

print("Test 1: ", modelTree.predict(test))
print("Test 2: ", modelTree.predict(test2))

0.941747572815534
1.0
Test 1:  ['Sub Legendary']
Test 2:  ['Mythical']


### Random Forest

In [57]:
modelRF = RandomForestClassifier(max_depth=10, random_state=0)
modelRF.fit(X_train,y_train)

y_predRF = modelRF.predict(X_test)

print(accuracy_score(y_test, y_predRF))
print(modelRF.score(X_train, y_train))

print("Test 1: ", modelRF.predict(test))
print("Test 2: ", modelRF.predict(test2))

0.9466019417475728
0.9466019417475728
Test 1:  ['Sub Legendary']
Test 2:  ['Mythical']


## ¿Cuál de los algoritmos podrá decir cual es el tipo correcto de los dos pokemon?

Decision tree y Random Forest

### Impriman los Accuracy de la siguiente manera para cada modelo

In [58]:
pd.DataFrame({"Type of Clasiffier":["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],"Accurancy on Train":[modelReg.score(X_train, y_train), modelSVM.score(X_train, y_train),modelTree.score(X_train, y_train),modelRF.score(X_train, y_train)],
    "Accurancy on Test":[modelReg.score(X_test, y_test), modelSVM.score(X_test, y_test),modelTree.score(X_test, y_test),modelRF.score(X_test, y_test)]})

,Type of Clasiffier,Accurancy on Train,Accurancy on Test
0,Logistic Regression,0.919708,0.898058
1,SVM,0.924574,0.902913
2,Decision Tree,1.000000,0.941748
3,Random Forest,1.000000,0.946602


### Probemos ahora los modelos!

### ¿Alguno mejora si aplicamos CV?

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
i = 1

# Recordemos que estamos ante un problema de desbalance de clases muy intenso!

In [59]:
pokedex.status.describe()

count       1028
unique         4
top       Normal
freq         915
Name: status, dtype: object